# In-class work 1: Introduction to training neural networks

Okay, now that we've covered some data science basics, let's get a very baseline neural network training pipeline going. 

**Important:** Make sure this notebook is pointing toward your custom `conda` kernel (top right drop-down menu).

<a name='section_0'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">0. Installing pytorch</h2>

In [1]:
%pip install torch torchvision

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 1.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 22.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 KB 24.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 22.3 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 15.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
### test that torch will work 

import torch

x = torch.rand(5, 3)
print(x)

tensor([[0.3929, 0.9546, 0.9154],
        [0.5970, 0.6793, 0.8816],
        [0.2915, 0.6950, 0.5424],
        [0.0468, 0.5761, 0.3842],
        [0.2035, 0.4470, 0.2181]])


Here's a full cell of imports. What's a good resolution we can use for plotting? Refer to the previous in-class notebook if you can't remember, and fill it in here:

In [ ]:
### here's the full cell of imports 

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

### set plot resolution
%config InlineBackend.figure_format = ??? # <<<<<<<<<<<< INSERT CODE HERE

### set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_1'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">1. Generating a dataset</h2>

First, we'll generate one of the simplest toy datasets there is: two Gaussian distributions. Let's say that they describe our signal and our background events, but that there are only 2 parameters we can measure ($\theta_1$ and $\theta_2$).

In [ ]:
### Two 2D Gaussian distributions with the same number of events
n_events = 2000

### Background is sampled from N(mu0, Sigma0)
mu0 = np.array([-1.0, -1.0])
cov0 = np.array([[1.0, 0.2],
                 [0.2, 1.2]])
x0 = np.random.multivariate_normal(mu0, cov0, size=n_events)
y0 = np.zeros((n_events, 1), dtype=np.float32)

### Signal is sampled from N(mu1, Sigma1)
mu1 = np.array([1.0, 1.0])
cov1 = np.array([[1.3, -0.3],
                 [-0.3, 0.8]])
x1 = np.random.multivariate_normal(mu1, cov1, size=n_events)
y1 = np.ones((n_events, 1), dtype=np.float32)

X = np.vstack([x0, x1]).astype(np.float32)
y = np.vstack([y0, y1]).astype(np.float32)

### Put these together and shuffle
idx = np.random.permutation(len(X))
X = X[idx] # inputs
y = y[idx] # labels

print("Data shapes:", X.shape, y.shape)

Let's put it into a Pandas dataframe for legibility:

In [ ]:
df = pd.DataFrame(np.hstack([X,y]), columns = ["theta_1", "theta_2", "label"])

In [ ]:
df

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(df[df.label != True].theta_1, df[df.label != True].theta_2, s=6, color="grey", alpha=0.5, label="Background")
plt.scatter(df[df.label == True].theta_1, df[df.label == True].theta_2, s=6, color="crimson", alpha=0.5, label="Signal")
plt.legend()
plt.title("Visualizing our datasets")
plt.xlabel(r"$\theta_1$"); plt.ylabel(r"$\theta_2$")
plt.show()

<a name='section_2'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">2. Defining a neural network model</h2>

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim=???, hidden_dim=32): # <<<< FILL IN INPUT DIMENSION 
        super().__init__()
        self.MLP = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)  # output is 1-dimensional for binary classification
        )
    def forward(self, x):
        return self.MLP(x)

<a name='section_3'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">3. Preprocessing</h2>

Key steps include splitting into train/validation/test sets, standardizing data, and building dataloaders.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["label"])  # only look at the features, not the labels
y = df["label"]                 # labels

### Take the first 70% for training
X_train, X_valtest, y_train, y_valtest = train_test_split(
    X, y, test_size=0.3)

### Then evenly split the remaining 30% into validation & test set 
X_val, X_test, y_val, y_test = train_test_split(
    X_valtest, y_valtest, test_size=0.5
)

print(f"Train set has {len(X_train)} events.")
print(f"Validation set has {len(X_val)} events.")
print(f"Test set has {len(X_test)} events.")

You can standardize your data using a "z-score" manually by subtracting the mean and dividing by the standard deviation. 

Alternatively, you can use the StandardScaler plugin to do this automatically:

In [ ]:
from sklearn.preprocessing import StandardScaler

### initialize the scaler
scaler = StandardScaler()

### IMPORTANT -- you only want to define your scaling ("fit_transform") based on your training dataset.
X_train_scaled = scaler.fit_transform(X_train)

### Now re-use the same transformation on validation & test sets ("transform")
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

print("Train mean (before scaling):", np.array(X_train.mean(axis=0)))
print("Train mean (after scaling) -- each dimension should be close to 0:", X_train_scaled.mean(axis=0))
print("Train std (before scaling)", np.array(X_train.std(axis=0)))
print("Train std (after scaling) -- each dimension should be close to 1:", X_train_scaled.std(axis=0))

<a name='problem_x'></a> 
### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #1f77b4; color: #1f77b4;">Problem: Standardizing with z-scores </span>

Try implementing this standardization by hand, and check that your scaled distributions are close to the ones that you got using the StandardScaler.

In [ ]:
X_train_scaled_by_hand = ??? 
X_val_scaled_by_hand = ??? 
X_test_scaled_by_hand = ??? 

In [ ]:
plt.figure()
bins = np.linspace(???, ??? , 40)
### theta_1
plt.hist(X_train_scaled[:,0], bins=bins, histtype="step", linewidth=2, label="$\theta_1$");
plt.hist(X_train_scaled_by_hand[:,0], bins=bins, histtype="step", linewidth=2, label="$\theta_1$ (by hand)");

In [ ]:
plt.figure()
bins = np.linspace(???, ??? , 40)
### theta_2
plt.hist(X_train_scaled[:,1], bins=bins, histtype="step", linewidth=2, label="$\theta_2$");
plt.hist(X_train_scaled_by_hand[:,1], bins=bins, histtype="step", linewidth=2, label="$\theta_2$ (by hand)";

In [ ]:
### now we'll construct data loaders. we'll use 32 as our default batch size for now. 

batch_size = 32

train_ds = TensorDataset(torch.from_numpy(X_train_scaled), torch.from_numpy(y_train.values.astype("float32")).view(-1, 1))
val_ds   = TensorDataset(torch.from_numpy(X_val_scaled),   torch.from_numpy(y_val.values.astype("float32")).view(-1, 1))
test_ds  = TensorDataset(torch.from_numpy(X_test_scaled),  torch.from_numpy(y_test.values.astype("float32")).view(-1, 1))

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

Test one of these loaders:

In [ ]:
for x_batch, y_batch in train_loader:
    print(x_batch.shape, y_batch.shape)
    break

<a name='section_4'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">4. Training</h2>

Let's play with tracking our model training using `livelossplot`.

In [ ]:
%pip install livelossplot

In [ ]:
from livelossplot import PlotLosses

In [ ]:
### Check for available GPUs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

### Initialize the model, optimizer, and loss function -- what should the input dimensionality be?
model = MLP(input_dim=???, hidden_dim=32).to(device) #<<<<<<<< FILL IN HERE
model = model.to(device) # move onto the GPU, if present
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=??? ) #<<<<<<<< FILL IN HERE -- what's the default learning rate for Adam?

liveloss = PlotLosses(figsize=(9, 4)) 
logs = {}

### Training loop
for epoch in range(10):

    ### train
    model.train()
    total_train_loss = 0
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        
        ### Move the batch onto the GPU, if present
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        out = model(x_batch)
        loss = loss_fn(out, y_batch)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    
    train_loss_per_batch = total_train_loss / len(train_loader)
    logs['loss'] = train_loss_per_batch

    ### validate
    model.eval()
    total_val_loss = 0
    for x_batch, y_batch in val_loader:
        
        ### Move the batch onto the GPU, if present
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        out = model(x_batch)
        loss = loss_fn(out, y_batch)
        total_val_loss += loss.item()
    
    val_loss_per_batch = total_val_loss / len(val_loader)
    logs['val_loss'] = val_loss_per_batch
    
    liveloss.update(logs)
    
    liveloss.send()
    print(f'Epoch {epoch}, Loss: {loss.item()}')

#### Using wandb

Skip this for now, but if you have time once you finish this notebook, try connecting the notebook to Wandb to have the option of tracking your runs: 
- https://docs.wandb.ai/guides/track/jupyter/ 

In [ ]:
# %pip install wandb -qqq

In [ ]:
# import wandb
# wandb.login()

In [ ]:
# wandb.init(
# ??? <<<< FILL IN HERE
#     },
# )

<a name='section_5'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">5. Evaluating model performance</h2>

The loss alone might not be a very intuitive metric for understanding our model performance. Let's evaluate the model on our holdout test set and get some new metrics.

In [ ]:
### set the threshold for your NN score (what value makes sense here? can you plot values of your NN scores to understand their distribution?) 
threshold = ??? # <<<< FILL IN HERE

### test set 
model.eval()
test_loss, test_acc, count = 0.0, 0.0, 0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        
        ### Move the batch onto the GPU, if present
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
    
        out = model(x_batch)
        loss = loss_fn(out, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_acc  += (torch.sigmoid(out) > threshold).float().eq(y_batch).float().mean().item() * x_batch.size(0)
        count += x_batch.size(0)
        
print(f"Average test loss: {test_loss/count:.4f} | Test accuracy: {test_acc/count:.3f}")

### Further exercises:
- Try modifying the hidden size and number of MLP layers, to see the effect on performance
- Try increasing the dataset size
- Try different learning rates
- What's the effect of changing your batch size?
- Try adding dropout
- Try making the task harder by making the Gaussians overlap and/or shifting their positions. Where does your model start to break down?
- Can you track an "accuracy" metric alongside the loss in your live tracking plots?
- Can you track your runs on Wandb? 
- What is a simple baseline you could define here to get context for how well your model is doing? Implement this and use it as a comparison.
- Can you visualize the decision boundary in parameter space?